In [1]:
import requests
import json
import pandas as pd

In [2]:
def createDF(resp):
    # Input is the "get" response for a Fund; return a formatted df with the dates and prices
    dtList = []
    valList = []
    for row in resp['data']:
        dtList.append(row['date'][:10])
        valList.append(row['close'])
    d = {}
    d['fund'] = [fund for x in range(len(dtList))]
    d['date'] = dtList
    d['price'] = valList
    return pd.DataFrame(d)

In [3]:
def getShares(df, fund):
    tmp = df.loc[fund]
    tmp.sort_values('date', inplace=True)
    return round(1000/tmp.iloc[0]['price'], 2)

In [4]:
# TODO
# Where is the data for DIISX?
# from_date only goes back 100 days

In [5]:
base = 'http://api.marketstack.com/v1/'
access_key = 'ffa68edbfb08de7589a43751795e0e07'

In [6]:
# End of day
url = base + 'eod'
url = url + '?access_key='+access_key

In [7]:
#date_from='2016-01-01'
#url = url + '&date_from='+date_from

In [8]:
funds = ['RYLD', 'ARKK', 'QYLD', 'NUSI', 'CCJ', 'COMB', 'JEPI', 'SPY']
#funds = ['RYLD', 'ARKK', 'QYLD', 'NUSI', 'CCJ', 'COMB', 'JEPI', 'DIISX.INDX', 'SPY']

In [9]:
dfList = []
for fund in funds:
    url = url + '&symbols=' + fund
    resp = requests.get(url).json()
    tmp = createDF(resp)
    dfList.append(tmp)

In [10]:
df = pd.concat(dfList)

In [12]:
# Now you have all the fund prices and dates
# Compute the number of shares $1000 would buy initially
df.set_index('fund', inplace=True)
shareList = []
for fund in funds:
    numShares = getShares(df, fund)
    shareList.append(numShares)
d = {}
d['fund']=funds
d['numShares'] = shareList

shares = pd.DataFrame(d)

<ipython-input-3-189b8613ca6e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.sort_values('date', inplace=True)


In [15]:
df = shares.join(df, on='fund', how='inner')

##### Compute fund values based on initial number of shares

In [21]:
df.loc[:, 'value'] = df['price']*df['numShares']
df['value'] = df['value'].astype('int')

In [27]:
df.to_csv('/home/tbrownex/data/retirement/funds/fundData.csv', index=False)